In [114]:
import os
import re
import openai
from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

import csv
# from csv import writer

In [115]:
load_dotenv()

True

In [116]:
# Secret keys from .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [117]:
# Directory Variables
RAW_TEXT_FILE = r"C:\Users\manas\Coding Playground\PythonP\TwitterBot\tweet_15.txt"
# TEST_TEXT_FILE = r"C:\Users\manas\Coding Playground\PythonP\TwitterBot\test_ideas.txt"
OUTPUT_CSV_FILE = r"C:\Users\manas\Coding Playground\PythonP\TwitterBot\tweets_final.csv"


# print variables
print(f"Raw Tweet File Dir: \n\t{RAW_TEXT_FILE}")
# print(f"Output CSV Dir: \n\t{TEST_TEXT_FILE}")
print(f"Test Dir: \n\t{OUTPUT_CSV_FILE}")

Raw Tweet File Dir: 
	C:\Users\manas\Coding Playground\PythonP\TwitterBot\tweet_15.txt
Test Dir: 
	C:\Users\manas\Coding Playground\PythonP\TwitterBot\tweets_final.csv


In [118]:
# This defines headers in the csv file.It resets the data, only run once. No need to run again and again
# def write_heaers_to_csv( csv_file_path):
#     headers = ["tweet", "topic", "explanation", "hashtags", "uploaded"]
#     with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
#         writer = csv.writer(file)
#         writer.writerow(headers)

In [119]:
# csv_file_path = OUTPUT_CSV_FILE  # Adjust path as needed
# write_heaers_to_csv(csv_file_path)

In [120]:
## Prompt Engineering with ChatGPT

In [121]:
llm = ChatOpenAI(temperature=0.3,
                 openai_api_key=OPENAI_API_KEY,
                 model_name='gpt-3.5-turbo-0613',
                )

In [122]:
def generate_response(
    llm: ChatOpenAI, coding_topic: str, coding_explaination: str
) -> tuple[str, str]:
    """Generate AI Twitter Content for CodingAunty Twitter Account

    Parameters:
        - llm:  pre-trained ChatOpenAi large language model
        - coding_topic: Topic related to Programming Languages
        - coding_explaination: Explanation of the coding topic

    Returns:
        - tuple[long response,short reposonse]: Chat GPT long and short responses
    """
    # System Template for LLM to follow
    system_template = """
        You are an incredibly wise and smart coder and programmer that lives and breathes the world of Programming Languages.
        Your goal is to writing short-form content for twitter given a `topic` in the area of Python programming  and a `explaination` from the user. There should be no :There should be no :🚀, 💪. 
        There should be no emojis or picture icons, incding but not limited to : 🔄,🤔, 🧠,😎, 🚀,💡,📚 or any other picture-icon
        There should be no tutorials, step-by-step instructions.

        % RESPONSE TONE:

        - Your response should be opinionated
        - No guides or step by step-bystep instructions or tutorial should be given
        - It should be one programming tip , for example, "remember to use indentation in python"
        - It can also be a defination, for example, "use websockets, a realtime technology that enables bidirectional, full-duplex communication between client and server over a persistent, single-socket connection."
        - Your tone should be serious
        
        % RESPONSE FORMAT:
        
        - Be extremely clear and concise
        - Respond with phrases no longer than two sentences
        - Do not respond with emojis
        
        % RESPONSE CONTENT:

        - Include specific examples of where this is used in the programming space
        - If you don't have an answer, say, "Sorry, I'll have to ask the Programming Gods!"    

        
        % RESPONSE TEMPLATE:

        - Here is the response structure: 
            Tweet: A captivating engaging tweet, hooking and engaging the user
            Hash Tags: 3 relevant hashtags
    
    """
    # system prompt template to follow
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

    # human template for input
    human_template = "topic to write about is {topic}, and the exolaination will be {explaination}. Keep the total response under 230 words total! No emojis should exist in the tweet or hash tags"
    human_message_prompt = HumanMessagePromptTemplate.from_template(
        human_template, input_variables=["topic", "explaination"]
    )

    # chat prompt template construction
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

    # get a completed chat using formatted template with topic and title
    final_prompt = chat_prompt.format_prompt(
        topic=coding_topic, explaination=coding_explaination
    ).to_messages()

    # pass template through llm and extract content attribute
    first_response = llm(final_prompt).content

    # # construct AI template, to pass back OpenAI response
    # ai_message_prompt = AIMessagePromptTemplate.from_template(first_response)

    #  # additional prompt to remind ChatGPT of length requirement
    # reminder_template = "This was good, but no guides and step by step or tutorials please.  but there are still picture Icons. Also Please remove any emojis or icons such as Remove any : 🔄,🤔, 🧠,😎,💡,📚 or any other picture-icon."
    # reminder_prompt = HumanMessagePromptTemplate.from_template(reminder_template)


    # # chat prompt template construction with additional AI response and length reminder
    # chat_prompt2 = ChatPromptTemplate.from_messages(
    #     [system_message_prompt, human_template, ai_message_prompt, reminder_prompt]
    # )

    # # get a completed chat using formatted template with topic and title
    # final_prompt = chat_prompt2.format_prompt(
    #     topic=coding_topic, explaination=coding_explaination
    # ).to_messages()

    # # pass template through llm and extract content attribute
    # short_response = llm(final_prompt).content

    return first_response



In [123]:
# # Test - Run to generate only one tweet
# first_response = generate_response(
#     llm, 
#     coding_topic='List Comprehensions', 
#     coding_explaination='Various ways to use list comprehensions for more concise and readable code.'
#     )

In [124]:
# first_response

In [125]:
# Function to extract the tweet and the hashtags
def extract_tweet_and_hashtags(first_response):
    # Split the data into two parts: Tweet and Hash Tags
    try:
        parts = first_response.split("\n\n")
        tweet = parts[0].replace('Tweet: "', '').replace('"', '').strip()
        hashtags = parts[1].replace('Hash Tags: ', '').strip().split(' ')
        hashtags_str = ', '.join(hashtags)
    except:
        tweet = first_response
        hashtags_str= ""
    return tweet, hashtags_str

In [126]:
# Take text file in table structure from GPT and process it by line divisors (|). 
# Then create dicitonary and store in desired format back to text file.

In [127]:
file = open(RAW_TEXT_FILE, "r")
python_tweets = {}

for line_no, line in enumerate(file.readlines()):
    # start 2nd row to avoid heading and underlines
    if line_no > 1:
        # split on line divisors
        items = line.split('|')
        # capture and ensure int and str formatting
        tweet_no = int(items[1])
        coding_topic = items[2].strip()
        coding_explaination = items[3].strip().strip('"')
        # store within dict
        python_tweets[tweet_no] = {}
        python_tweets[tweet_no]['topic'] = coding_topic
        python_tweets[tweet_no]['explaination'] = coding_explaination
        # print tweet no, topic and title
        print(f"{tweet_no}_{coding_topic}_{coding_explaination}")
file.close()

1_Personal Budget Tracker_Develop an application that allows users to track their expenses and income, categorize them, and visualize spending trends.
2_Weather Forecast App_Create a weather app that fetches data from a weather API, providing forecasts, alerts, and climate trends.
3_Custom Recipe Manager_Build a recipe management system where users can add, edit, and organize recipes, including ingredient lists and instructions.
4_Fitness Activity Tracker_Develop a fitness tracker that logs activities, calculates calories burned, and tracks progress over time.
5_Language Learning Assistant_Create a language learning app with interactive lessons, quizzes, and progress tracking for various languages.
6_Personal Diary Application_Build a secure, private digital diary app with features like text entries, photo uploads, and mood tracking.
7_Online Survey Tool_Develop an online survey platform that allows users to create surveys, collect responses, and analyze results.
8_Task and Project Man

In [128]:
for tweet_no,tweet_info in python_tweets.items():
    # print(tweet_info['topic'])
    # print(tweet_info['explaination'])
    # # first_reponse = (tweet_info['topic'] + ':' + tweet_info['explaination']+ '\n')
    # # print(first_response)
    final_reponse = generate_response(
        llm, 
        coding_topic= tweet_info['topic'], 
        coding_explaination=tweet_info['explaination']
    )
    tweet, hashtags_str = extract_tweet_and_hashtags(final_reponse)
    print(f"{tweet_no}_{tweet}")
    Data = [tweet, tweet_info['topic'], tweet_info['explaination'], hashtags_str, 0]
    
    with open(OUTPUT_CSV_FILE, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(Data)
        # Close the file object
        file.close()

    

1_Manage your finances with ease! 📊💰 Develop a Personal Budget Tracker app to effortlessly track expenses, categorize income, and visualize spending trends. Stay on top of your financial game! #PersonalBudgetTracker #FinanceManagement #TrackExpenses
2_When building a Weather Forecast App, make sure to choose a reliable weather API that provides accurate and up-to-date data. #codingproject #weatherapp #API
3_Tweet: 
Looking to level up your cooking game? 🍳👨‍🍳 Build a custom recipe manager to effortlessly organize and edit your favorite recipes. Say goodbye to messy cookbooks and hello to a digital kitchen assistant! #RecipeManager #CookingHacks #CodingProject
4_Looking to stay fit? Develop a fitness tracker that logs activities, calculates calories burned, and tracks progress over time. #FitnessTracker #HealthApp #CodingProject
5_Boost your language skills with a cutting-edge Language Learning Assistant! 🌐📚🎓 Engage in interactive lessons, quizzes, and track your progress effortlessly. #

KeyboardInterrupt: 